In [11]:
import uproot
import pandas as pd

file = uproot.open("KKpi_flat_bestX2_2017.root")
tree = file["pipkmks__B4_M16"]
print(tree.keys())
df = pd.DataFrame()

print(df.columns)


['run', 'event', 'trigbits', 'rftime', 'kin_chisq', 'kin_ndf', 'beam_beamid', 'beam_x4_meas', 'beam_p4_meas', 'beam_x4_kin', 'beam_p4_kin', 'pip1_trkid', 'pip1_x4_meas', 'pip1_p4_meas', 'pip1_x4_kin', 'pip1_p4_kin', 'pip1_pid_fom', 'pip1_beta_time', 'pip1_chisq_time', 'pip1_ndf_time', 'pip1_ndf_trk', 'pip1_chisq_trk', 'pip1_ndf_dedx', 'pip1_chisq_dedx', 'pip1_dedx_cdc', 'pip1_dedx_fdc', 'pip1_dedx_tof', 'pip1_dedx_st', 'pip1_ebcal', 'pip1_eprebcal', 'pip1_efcal', 'pip1_bcal_delphi', 'pip1_bcal_delz', 'pip1_fcal_doca', 'km_trkid', 'km_x4_meas', 'km_p4_meas', 'km_x4_kin', 'km_p4_kin', 'km_pid_fom', 'km_beta_time', 'km_chisq_time', 'km_ndf_time', 'km_ndf_trk', 'km_chisq_trk', 'km_ndf_dedx', 'km_chisq_dedx', 'km_dedx_cdc', 'km_dedx_fdc', 'km_dedx_tof', 'km_dedx_st', 'km_ebcal', 'km_eprebcal', 'km_efcal', 'km_bcal_delphi', 'km_bcal_delz', 'km_fcal_doca', 'p_trkid', 'p_x4_meas', 'p_p4_meas', 'p_x4_kin', 'p_p4_kin', 'p_pid_fom', 'p_beta_time', 'p_chisq_time', 'p_ndf_time', 'p_ndf_trk', 'p_chi

In [3]:
import pandas as pd
import numpy as np
import uproot

def analysis(filename, treename):
    # Open the ROOT file using uproot
    with uproot.open(filename) as file:
        # Read the necessary branches as arrays
        branches = file[treename].arrays(['pip2_E', 'pip2_px', 'pip2_py', 'pip2_pz', 'pim_E', 'pim_px', 'pim_py', 'pim_pz'], library="np")
        
        # Create an empty DataFrame
        df = pd.DataFrame()
        
        # Perform calculations and add as new columns
        df['pip2pim_E'] = branches['pip2_E'] + branches['pim_E']
        df['ks_E'] = df['pip2pim_E']  # Alias
        
        df['pip2pim_px'] = branches['pip2_px'] + branches['pim_px']
        df['ks_px'] = df['pip2pim_px']  # Alias
        
        df['pip2pim_py'] = branches['pip2_py'] + branches['pim_py']
        df['ks_py'] = df['pip2pim_py']  # Alias
        
        df['pip2pim_pz'] = branches['pip2_pz'] + branches['pim_pz']
        df['ks_pz'] = df['pip2pim_pz']  # Alias
        
        # Calculate total mass
        df['pip2pim_m'] = np.sqrt(df['pip2pim_E']**2 - df['pip2pim_px']**2 - df['pip2pim_py']**2 - df['pip2pim_pz']**2)
        df['ks_m'] = df['pip2pim_m']  # Alias

        # Print dataframe columns to check the new ones
        print(df.columns)

# Replace the filename and treename with your actual file and tree names
analysis("KKpi_flat_bestX2_2017.root", "pipkmks__B4_M16;1")


Index(['pip2pim_E', 'ks_E', 'pip2pim_px', 'ks_px', 'pip2pim_py', 'ks_py',
       'pip2pim_pz', 'ks_pz', 'pip2pim_m', 'ks_m'],
      dtype='object')


In [8]:
%env ROOTSYS=/group/halld/Software/builds/Linux_CentOS7.7-x86_64-gcc4.8.5/root/root-6.24.04
%env PYTHONPATH=/group/halld/Software/builds/Linux_CentOS7.7-x86_64-gcc4.8.5/root/root-6.24.04/lib:$PYTHONPATH
%env LD_LIBRARY_PATH=/group/halld/Software/builds/Linux_CentOS7.7-x86_64-gcc4.8.5/root/root-6.24.04/lib:$LD_LIBRARY_PATH

import ROOT

env: ROOTSYS=/group/halld/Software/builds/Linux_CentOS7.7-x86_64-gcc4.8.5/root/root-6.24.04
env: PYTHONPATH=/group/halld/Software/builds/Linux_CentOS7.7-x86_64-gcc4.8.5/root/root-6.24.04/lib:$PYTHONPATH
env: LD_LIBRARY_PATH=/group/halld/Software/builds/Linux_CentOS7.7-x86_64-gcc4.8.5/root/root-6.24.04/lib:$LD_LIBRARY_PATH


ModuleNotFoundError: No module named 'ROOT'

In [11]:
%env ROOTSYS=/group/halld/Software/builds/Linux_CentOS7.7-x86_64-gcc4.8.5/root/root-6.24.04
%env PYTHONPATH=/group/halld/Software/builds/Linux_CentOS7.7-x86_64-gcc4.8.5/root/root-6.24.04/lib:$PYTHONPATH
%env LD_LIBRARY_PATH=/group/halld/Software/builds/Linux_CentOS7.7-x86_64-gcc4.8.5/root/root-6.24.04/lib:$LD_LIBRARY_PATH
import ROOT
import uproot
import numpy as np

def analysis(filename, treename):
    # Open the ROOT file using uproot
    with uproot.open(filename) as file:
        # Extract arrays for the required branches
        branches = file[treename].arrays(['pip2_E', 'pip2_px', 'pip2_py', 'pip2_pz', 'pim_E', 'pim_px', 'pim_py', 'pim_pz'], library="np")

        # Calculate the quantities before creating the DataFrame
        pip2pim_E = branches['pip2_E'] + branches['pim_E']
        pip2pim_px = branches['pip2_px'] + branches['pim_px']
        pip2pim_py = branches['pip2_py'] + branches['pim_py']
        pip2pim_pz = branches['pip2_pz'] + branches['pim_pz']
        pip2pim_m = np.sqrt(pip2pim_E**2 - pip2pim_px**2 - pip2pim_py**2 - pip2pim_pz**2)

        # Instantiate a TH1F histogram filled with values of "ks_m"
        # Create a TH1F object
        h_ks_m = ROOT.TH1F("h_ks_m", "Invariant Mass of K_S; Mass (GeV/c^2); Counts", 50, min(pip2pim_m), max(pip2pim_m))
        
        # Fill the histogram
        for value in pip2pim_m:
            h_ks_m.Fill(value)

        # Draw the histogram
        h_ks_m.Draw()

        # Note: To see the histogram, you'll need to run this in a ROOT-enabled environment,
        # or use a Jupyter notebook with ROOT integration.

# Replace the filename and treename with your actual file and tree names
analysis("KKpi_flat_bestX2_2017.root", "pipkmks__B4_M16;1")


env: ROOTSYS=/group/halld/Software/builds/Linux_CentOS7.7-x86_64-gcc4.8.5/root/root-6.24.04
env: PYTHONPATH=/group/halld/Software/builds/Linux_CentOS7.7-x86_64-gcc4.8.5/root/root-6.24.04/lib:$PYTHONPATH
env: LD_LIBRARY_PATH=/group/halld/Software/builds/Linux_CentOS7.7-x86_64-gcc4.8.5/root/root-6.24.04/lib:$LD_LIBRARY_PATH


ModuleNotFoundError: No module named 'ROOT'